In [3]:
!git clone https://github.com/FleshRazer/F22-Introduction-to-Machine-Learning.git

Cloning into 'F22-Introduction-to-Machine-Learning'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 114 (delta 30), reused 90 (delta 18), pack-reused 0
Receiving objects: 100% (114/114), 1.96 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer, load_digits
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Naive Bayes

In [4]:
data = pd.read_csv('F22-Introduction-to-Machine-Learning/Lab-3/titanic.csv')
# removing name column
data = data.drop(['name'], axis = 1)
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.00,0,0,211.3375,S
1,1,1,male,0.92,1,2,151.5500,S
2,0,1,female,2.00,1,2,151.5500,S
3,0,1,male,30.00,1,2,151.5500,S
4,0,1,female,25.00,1,2,151.5500,S


In [5]:
# split data
x_train, x_test, y_train, y_test = train_test_split(data.loc[:, 'pclass':], data['survived'],
                                                    test_size=0.2, stratify=data['survived'])

## Data preprocessing
NB can handle discrete features data which can be useful with categorical data.

Let's see one of the advantages of NB classifier. NB is not affected by data scaling

In [6]:
# imputing missing values
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(x_train)
x_train = pd.DataFrame(imputer.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_test.columns)

# one-hot-encode categorical features
def ohe_new_features(df, features_name, encoder):
    new_feats = encoder.transform(df[features_name])
    # create dataframe from encoded features with named columns
    new_cols = pd.DataFrame(new_feats, dtype=int, columns=encoder.get_feature_names(features_name))
    new_df = pd.concat([df, new_cols], axis=1)
    new_df.drop(features_name, axis=1, inplace=True)
    return new_df

encoder = OneHotEncoder(sparse=False, drop='first')
f_names = ['sex', 'embarked']
encoder.fit(x_train[f_names])
x_train = ohe_new_features(x_train, f_names, encoder)
x_test = ohe_new_features(x_test, f_names, encoder)

# feature scaling
scaler = MinMaxScaler()
scaler.fit(x_train)
scaled_x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
scaled_x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train and test two NB models ono the data before scaling and one after scaling
and observe if the accuracy change with scaling

In [13]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB


models = {'GaussianNB': GaussianNB(), 'MultinomialNB': MultinomialNB()}

# Write code here
for name, nb in models.items():
    nb.fit(x_train, y_train)
    y_pred = nb.predict(x_test)
    print(f'{name} accuracy before scaling: {accuracy_score(y_test, y_pred)}')

# Write code here
for name, nb in models.items():
    nb.fit(scaled_x_train, y_train)
    y_pred = nb.predict(scaled_x_test)
    print(f'{name} accuracy before scaling: {accuracy_score(y_test, y_pred)}')

GaussianNB accuracy before scaling: 0.7748091603053435
MultinomialNB accuracy before scaling: 0.6526717557251909
GaussianNB accuracy before scaling: 0.7748091603053435
MultinomialNB accuracy before scaling: 0.6526717557251909


```
Accuracy before scaling: 0.7633587786259542
Accuracy after scaling: 0.7633587786259542
```

# Regularization
What is [Elastic-Net](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net)?
How can you specify the contribution of each part using l1 ration

Apply classification on the breast cancer dataset with no regularization, l1,
l2, and elastic-net.

### Loading Breast cancer dataset

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

### Fitting both Lasso and Ridge

Fit 3 models: Lasso and Ridge and Elastic-Net.
Then print their accuracy and coefficients and notice the difference.

In [29]:
from sklearn.linear_model import LogisticRegression


lasso = LogisticRegression(penalty='l1', solver='liblinear')
lasso.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print('Lasso', classification_report(y_test, y_pred))
print("Lasso Coefficient:", *lasso.coef_, sep='\n\t')
print('Sum of lasso abs values:', np.sum(np.abs(lasso.coef_)))

Lasso               precision    recall  f1-score   support

           0       0.81      0.83      0.82       162
           1       0.71      0.68      0.69       100

    accuracy                           0.77       262
   macro avg       0.76      0.75      0.76       262
weighted avg       0.77      0.77      0.77       262

Lasso Coefficient:
	[-0.83470541 -0.03538166 -0.29065175 -0.07829262  0.00258102 -2.55322452
 -0.49444722 -0.46665748]
Sum of lasso abs values: 4.755941679744645


In [27]:
ridge = LogisticRegression(penalty='l2', solver='liblinear')
ridge.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print('Ridge', classification_report(y_test, y_pred))
print("Ridge Coefficient:", *ridge.coef_, sep='\n\t')
print('Sum of ridge abs values:', np.sum(np.abs(ridge.coef_)))

Ridge               precision    recall  f1-score   support

           0       0.81      0.83      0.82       162
           1       0.71      0.68      0.69       100

    accuracy                           0.77       262
   macro avg       0.76      0.75      0.76       262
weighted avg       0.77      0.77      0.77       262

Ridge Coefficient:
	[-0.70110315 -0.02962878 -0.27788428 -0.08214945  0.00376451 -2.43960123
 -0.50143228 -0.42100097]
Sum of ridge abs values: 4.456564655023065


In [28]:
elasticnet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
elasticnet.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print('ElasticNet', classification_report(y_test, y_pred))
print("Ridge Coefficient:", *elasticnet.coef_, sep='\n\t')
print('Sum of ridge abs values:', np.sum(np.abs(elasticnet.coef_)))

ElasticNet               precision    recall  f1-score   support

           0       0.81      0.83      0.82       162
           1       0.71      0.68      0.69       100

    accuracy                           0.77       262
   macro avg       0.76      0.75      0.76       262
weighted avg       0.77      0.77      0.77       262

Ridge Coefficient:
	[-0.07720081 -0.02131138 -0.03739339 -0.00436155  0.01270135 -0.0894325
 -0.00012684 -0.02315032]
Sum of ridge abs values: 0.26567813244391314


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


# KNN
Compare KNN vs logistic regression

---
In ML images can be flattened to 1D vector of pixels, then we can train our
models on them considering each pixel as a feature.

Dataset shape (1797, 8, 8)


Now we have 1797 images flattened to 64-values vectors


In [30]:
# Based on https://github.com/hsu-ai-course/hsu.ai/blob/master/code/12.%20kNN%20and%20ANN%20for%20MNIST.ipynb
digits = load_digits()

print("Dataset shape", digits.images.shape)

# show first image
plt.title(f"Target: {digits.target[0]}")
plt.imshow(digits.images[0], cmap='gray')
plt.show()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target
print("Now we have {} images flattened to {}-values vectors".format(*X.shape))

In [31]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Train a KNN and LR models and compare their results

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Write code here
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

LR = LogisticRegression()
LR.fit(X_train, y_train)

y_true, y_pred = y_test, knn.predict(X_test)
print('KNN', classification_report(y_true, y_pred))

y_true, y_pred = y_test, LR.predict(X_test)
print('LR', classification_report(y_true, y_pred))

KNN               precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       0.97      0.97      0.97        35
           2       1.00      0.97      0.99        36
           3       0.91      1.00      0.95        29
           4       1.00      0.97      0.98        30
           5       0.95      0.97      0.96        40
           6       1.00      1.00      1.00        44
           7       0.95      1.00      0.97        39
           8       1.00      0.90      0.95        39
           9       0.98      0.98      0.98        41

    accuracy                           0.97       360
   macro avg       0.98      0.98      0.98       360
weighted avg       0.98      0.97      0.97       360

LR               precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       0.92      0.97      0.94        35
           2       0.97      0.97      0.97        36
           3     

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Which model performed better? What is the advantages of each model over the other?

What is the output of `classification_report` function? How to interpret it?